In [0]:
#@title Setup Environment
#@test {"output": "ignore"}

import glob

print 'Copying checkpoints and example MIDI from GCS. This will take a few minutes...'
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/* /content/

print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print 'Importing libraries and defining some helper functions...'
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow as tf

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print 'Start Seq Reconstruction'
  play(note_sequences[0])
  print 'End Seq Reconstruction'
  play(note_sequences[-1])
  print 'Mean Sequence'
  play(note_sequences[num_steps // 2])
  print 'Start -> End Interpolation'
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print 'Done'

In [0]:
#@title Load the pre-trained model.
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/mel_2bar_big.ckpt')

In [0]:
genValues = [[0.03125, -0.6875, 0.0, -0.1875, -0.34375, 0.09375, 0.53125, -0.3125, -0.625, 0.59375, -0.5, -0.90625, -0.125, 0.6875, -0.375, 0.9375, 0.125, 0.3125, -0.5, 0.3125, -0.125, -0.15625, -0.625, -0.6875, 0.6875, 0.09375, 0.65625, -0.09375, -1.0, -0.375, -0.03125, 0.4375, 0.0, -0.59375, 0.46875, -0.6875, -0.96875, -0.21875, 0.4375, -0.8125, 0.28125, -0.4375, 0.625, 0.40625, 0.15625, 0.40625, -0.21875, 0.875, -0.125, -0.65625, -0.65625, -0.75, 0.46875, 0.875, 0.75, -0.59375, 0.5625, 0.0, -0.40625, 0.5625, 0.6875, -0.0625, 0.625, 0.625, -0.21875, -0.40625, 0.40625, 0.71875, -0.15625, 0.09375, 0.71875, -0.78125, -0.125, -0.75, -1.0, -0.65625, -0.03125, -0.21875, -0.8125, 0.8125, 0.5, 0.3125, -1.0, 0.5625, 0.0625, -0.84375, -0.53125, 0.65625, -0.9375, 0.0625, 0.46875, -0.34375, -0.96875, -0.84375, 0.6875, -0.3125, 0.0, -0.8125, 0.84375, -0.625, 0.9375, 0.53125, 0.75, -0.125, 0.21875, -0.5625, -0.5625, -0.5, 0.125, -0.75, -0.59375, 0.875, -0.40625, -0.6875, -0.53125, 0.3125, 0.4375, 0.625, -0.15625, 0.28125, -0.96875, 0.125, 0.21875, -0.8125, -1.0, -0.5625, -0.5625, 0.125, 0.5, -0.09375, 0.46875, 0.09375, 0.53125, 0.65625, 0.53125, 0.6875, -0.59375, -0.0625, 0.9375, -0.5625, 0.71875, 0.5, -0.34375, -0.34375, -0.15625, 0.75, 0.03125, -0.65625, 0.25, -0.6875, -0.96875, 0.34375, 0.59375, 0.625, 0.53125, -0.75, 0.1875, 0.78125, -0.53125, -0.40625, 0.75, -0.53125, 0.84375, 0.6875, 0.90625, -0.40625, 0.75, -0.0625, 0.625, -0.84375, 0.6875, 0.90625, -0.15625, -0.28125, -0.21875, -0.8125, -0.15625, -0.09375, 0.65625, 0.625, -1.0, 0.875, -0.875, 0.75, 0.53125, -0.96875, -0.5, -0.84375, 0.40625, 0.84375, 0.4375, -0.6875, -0.0625, 0.1875, -0.78125, 0.75, -0.96875, -0.65625, 0.71875, -0.25, 0.1875, 0.28125, -0.3125, -0.125, -0.5, 0.1875, 0.0, -0.09375, 0.65625, -0.34375, -0.1875, 0.65625, 0.0625, 0.4375, -0.6875, 0.375, 0.34375, -0.4375, -0.34375, 0.8125, 0.9375, 0.125, 0.875, 0.90625, -0.21875, 0.0, -0.28125, 0.75, 0.34375, -0.0625, 0.09375, -0.09375, 0.78125, 0.9375, 0.8125, 0.875, 0.5625, 0.875, -0.6875, 0.53125, -0.28125, 0.71875, -0.375, 0.3125, -0.28125, 0.84375, -0.84375, -1.0, 0.625, 0.0, -0.09375, -0.5, -0.0625, 0.9375, 0.8125, -0.8125, -0.90625, -0.3125, 0.6875, -0.96875, 0.34375, -0.53125, 0.15625, -0.09375, 0.625, 0.40625, 0.4375, 0.125, -0.125, -0.46875, 0.53125, -0.90625, -0.9375, 0.03125, 0.71875, -0.40625, -0.6875, 0.9375, 0.96875, 0.21875, -0.28125, 0.21875, 0.9375, 0.96875, -0.4375, 0.5625, 0.65625, -0.71875, -0.5, 0.8125, 0.78125, -0.09375, 0.0, 0.6875, 0.84375, 0.8125, -0.84375, 0.84375, -0.09375, -0.875, -0.34375, 0.0625, 0.5625, 0.53125, 0.5625, -0.75, -0.21875, 0.53125, 0.625, -0.21875, 0.78125, 0.0625, -1.0, -0.03125, 0.5625, -0.8125, -0.34375, -0.75, -0.21875, 0.9375, 0.125, -0.9375, -0.75, -0.09375, -0.625, -0.71875, -0.0625, 0.375, 0.15625, -0.375, 0.09375, 0.25, 0.5625, 0.375, -0.96875, -0.75, 0.90625, 0.5, 0.9375, -0.71875, 0.3125, 0.875, 0.5625, 0.0625, -0.40625, 0.90625, 0.28125, -0.65625, -0.125, 0.71875, 0.71875, 0.9375, -0.3125, 0.4375, -0.34375, -0.8125, 0.3125, -0.25, -0.625, -0.0625, -0.34375, 0.0625, 0.875, -0.28125, 0.34375, -0.09375, 0.28125, 0.78125, -0.625, -0.40625, 0.09375, -0.5625, -0.65625, 0.5, 0.8125, 0.9375, 0.625, 0.59375, 0.59375, -0.625, -0.90625, -0.59375, -0.90625, 0.0, 0.59375, 0.40625, 0.34375, -0.8125, 0.34375, 0.40625, 0.96875, -0.3125, 0.40625, 0.53125, 0.1875, -0.59375, -0.28125, -0.96875, 0.3125, 0.40625, 0.65625, -0.03125, 0.96875, -0.40625, -0.03125, -0.25, 0.5625, 0.8125, -0.78125, -0.0625, -0.0625, 0.09375, 0.1875, -0.4375, -0.21875, 0.375, 0.3125, 0.5, -0.84375, 0.0, 0.15625, 0.09375, 0.40625, -0.75, -0.53125, -0.28125, 0.21875, 0.8125, -0.125, -0.53125, -0.875, -0.625, -0.75, -0.15625, 0.34375, 0.78125, -0.59375, -0.6875, 0.46875, -1.0, 0.15625, -0.25, -0.625, -1.0, 0.78125, 0.25, -0.59375, 0.0625, 0.84375, 0.9375, -0.3125, -0.96875, 0.53125, 0.75, 0.40625, 0.125, -0.375, -0.625, 0.75, -0.96875, 0.59375, 0.09375, 0.8125, 0.21875, 0.375, 0.4375, 0.59375, -0.15625, -0.1875, 0.5625, 0.125, -0.65625, 0.5, -0.65625, 0.21875, 0.90625, 0.59375, 0.78125, -0.40625, -0.65625, 0.96875, 0.15625, -0.59375, 0.625, 0.75, -0.25, -0.1875, -0.28125, -0.46875, -0.1875, 0.625, -0.03125, -0.6875, 0.375, 0.8125, -0.875, -0.6875, -0.9375, 0.8125, -0.625, 0.6875, 0.09375, -0.4375, -0.65625, -0.21875, -0.40625, -0.75, 0.0, -0.21875, -0.78125, 0.40625, -0.78125], [0.125, -0.59375, -0.875, -0.21875, -0.875, 0.03125, -0.78125, 0.5, -0.15625, 0.53125, -0.34375, -0.625, 0.3125, 0.71875, 0.65625, 0.4375, -0.5, 0.15625, 0.1875, 0.75, 0.03125, 0.8125, -0.5625, -0.5, 0.84375, -0.28125, -0.9375, -0.1875, -0.75, -0.90625, -0.78125, 0.34375, 0.9375, 0.625, -0.28125, 0.1875, 0.96875, -0.625, 0.21875, 0.46875, 0.0625, 0.4375, -0.75, 0.6875, 0.21875, -0.78125, -0.09375, 0.5, 0.15625, -0.90625, -0.28125, 0.4375, 0.96875, 0.5, 0.875, 0.09375, 0.46875, 0.46875, 0.875, -0.40625, -0.3125, 0.5625, 0.21875, 0.5, -0.09375, -0.5625, -0.78125, 0.15625, 0.28125, -0.96875, 0.21875, -0.8125, 0.96875, 0.65625, -0.6875, -0.28125, -0.8125, 0.84375, -0.875, 0.375, -0.96875, -0.21875, -0.1875, 0.46875, 0.71875, -0.8125, 0.84375, 0.3125, 0.875, 0.8125, -0.15625, 0.6875, -0.40625, -0.84375, 0.71875, 0.125, 0.125, -0.4375, 0.125, -0.3125, -0.59375, -0.75, 0.8125, -0.28125, 0.28125, 0.5, 0.03125, 0.25, 0.71875, 0.15625, 0.3125, -0.125, -0.65625, -0.75, 0.96875, -0.78125, 0.25, 0.25, 0.75, -0.15625, 0.46875, 0.78125, -0.21875, -0.0625, 0.375, 0.09375, 0.3125, 0.53125, -0.46875, 0.4375, -0.3125, -0.28125, 0.03125, -0.6875, 0.75, -0.96875, 0.125, 0.59375, 0.78125, 0.59375, -0.875, -0.3125, 0.125, 0.9375, 0.09375, 0.28125, 0.375, 0.15625, -0.65625, 0.8125, 0.53125, -0.53125, -0.375, -0.125, -0.625, -0.6875, 0.46875, 0.96875, -0.1875, 0.0, 0.5, -0.90625, -0.71875, 0.25, 0.84375, -0.8125, -0.65625, -0.78125, 0.65625, 0.28125, 0.34375, 0.46875, -0.6875, -0.375, -0.875, -0.59375, -0.5, 0.65625, 0.375, 0.65625, -0.375, 0.90625, -0.6875, -0.875, -0.46875, -0.53125, -0.40625, 0.375, -0.25, -0.71875, 0.4375, -0.375, 0.65625, -1.0, -0.21875, 0.46875, -0.5625, 0.0625, -0.5625, 0.15625, 0.21875, -0.90625, 0.59375, 0.8125, 0.9375, -0.875, 0.53125, -0.96875, -0.03125, -0.09375, 0.65625, 0.8125, 0.6875, -0.25, -0.0625, -0.53125, -0.15625, -0.375, -1.0, -0.28125, -0.875, 0.9375, 0.15625, -0.125, -0.125, -0.4375, 0.25, 0.09375, 0.75, 0.0625, -0.3125, -0.9375, 0.625, -0.09375, -0.46875, -0.65625, -0.28125, -0.1875, 0.1875, -0.4375, 0.3125, 0.84375, -1.0, 0.71875, -0.625, -0.21875, 0.53125, -0.34375, -0.75, 0.28125, -0.875, 0.28125, 0.625, -0.0625, -0.3125, -0.8125, -0.90625, -0.3125, 0.6875, -0.96875, 0.34375, -0.53125, 0.15625, -0.09375, 0.625, 0.40625, 0.4375, 0.125, -0.125, -0.46875, 0.53125, -0.90625, -0.9375, 0.03125, 0.71875, -0.40625, -0.6875, 0.9375, 0.96875, 0.21875, -0.28125, 0.21875, 0.9375, 0.96875, -0.4375, 0.5625, 0.65625, -0.71875, -0.5, 0.8125, 0.78125, -0.09375, 0.0, 0.6875, 0.84375, 0.8125, -0.84375, 0.84375, -0.09375, -0.875, -0.34375, 0.0625, 0.5625, 0.53125, 0.5625, -0.75, -0.21875, 0.53125, 0.625, -0.21875, 0.78125, 0.0625, -1.0, -0.03125, 0.5625, -0.8125, -0.34375, -0.75, -0.21875, 0.9375, 0.125, -0.9375, -0.75, -0.09375, -0.625, -0.71875, -0.0625, 0.375, 0.15625, -0.375, 0.09375, 0.875, 0.5625, 0.375, -0.96875, -0.75, 0.90625, 0.5, 0.9375, -0.71875, 0.3125, 0.875, 0.5625, 0.0625, -0.40625, 0.90625, 0.28125, -0.65625, -0.125, 0.71875, 0.71875, 0.9375, -0.3125, 0.4375, -0.34375, -0.8125, 0.3125, -0.25, -0.625, -0.0625, -0.34375, 0.0625, 0.875, -0.28125, 0.34375, -0.09375, 0.28125, 0.78125, -0.625, -0.40625, 0.09375, -0.5625, -0.65625, 0.5, 0.8125, 0.9375, 0.625, 0.59375, 0.59375, -0.625, -0.90625, -0.59375, -0.90625, 0.0, 0.59375, 0.40625, 0.34375, -0.8125, 0.34375, 0.40625, 0.96875, -0.3125, 0.40625, 0.53125, 0.1875, -0.59375, -0.28125, -0.96875, 0.3125, 0.40625, 0.65625, -0.03125, 0.96875, -0.40625, -0.03125, -0.25, 0.5625, 0.8125, -0.78125, -0.0625, -0.0625, 0.09375, 0.1875, -0.4375, -0.21875, 0.375, 0.3125, 0.5, -0.84375, 0.0, 0.15625, 0.09375, 0.40625, -0.75, -0.53125, -0.28125, 0.21875, 0.8125, -0.125, -0.53125, -0.875, -0.625, -0.75, -0.15625, 0.34375, 0.78125, -0.59375, -0.6875, 0.46875, -1.0, 0.15625, -0.25, -0.625, -1.0, 0.78125, 0.25, -0.59375, 0.0625, 0.84375, 0.9375, -0.3125, -0.96875, 0.53125, 0.75, 0.40625, 0.125, -0.375, -0.625, 0.75, -0.96875, 0.59375, 0.09375, 0.8125, 0.21875, 0.375, 0.4375, 0.59375, -0.15625, -0.1875, 0.5625, 0.125, -0.65625, 0.5, -0.65625, 0.21875, 0.90625, 0.59375, 0.78125, -0.40625, -0.65625, 0.96875, 0.15625, -0.59375, 0.625, 0.75, -0.25, -0.1875, -0.28125, -0.46875, -0.1875, 0.625, -0.03125, -0.6875, 0.375, 0.8125, -0.875, -0.6875, -0.9375, 0.8125, -0.625, 0.6875, 0.09375, -0.4375, -0.65625, -0.21875, -0.40625, -0.75, 0.0, -0.21875, -0.78125, 0.40625, -0.78125], [0.125, -0.59375, -0.875, -0.21875, -0.875, 0.03125, -0.78125, 0.5, -0.15625, 0.53125, -0.34375, -0.625, 0.3125, 0.71875, 0.65625, 0.4375, -0.5, 0.15625, 0.1875, 0.75, 0.03125, 0.8125, -0.5625, -0.5, 0.84375, -0.28125, -0.9375, -0.1875, -0.75, -0.90625, -0.78125, 0.34375, 0.9375, 0.625, -0.28125, 0.1875, 0.96875, -0.625, 0.21875, 0.46875, 0.0625, 0.4375, -0.75, 0.6875, 0.21875, -0.78125, -0.09375, 0.5, 0.15625, -0.90625, -0.28125, 0.4375, 0.96875, 0.5, 0.875, 0.09375, 0.46875, 0.46875, 0.875, -0.40625, -0.3125, 0.5625, 0.21875, 0.5, -0.09375, -0.5625, -0.78125, 0.15625, 0.28125, -0.96875, 0.21875, -0.8125, 0.96875, 0.65625, -0.6875, -0.28125, -0.8125, 0.84375, -0.875, 0.375, -0.96875, -0.21875, -0.1875, 0.46875, 0.71875, -0.8125, 0.84375, 0.3125, 0.875, 0.8125, -0.15625, 0.6875, -0.40625, -0.84375, 0.71875, 0.125, 0.125, -0.4375, 0.125, -0.3125, -0.59375, -0.75, 0.8125, -0.28125, 0.28125, 0.5, 0.03125, 0.25, 0.71875, 0.15625, 0.3125, -0.125, -0.65625, -0.75, 0.96875, -0.78125, 0.25, 0.25, 0.75, -0.15625, 0.46875, 0.78125, -0.21875, -0.0625, 0.375, 0.09375, 0.3125, 0.53125, -0.46875, 0.4375, -0.3125, -0.28125, 0.03125, -0.6875, 0.75, -0.96875, 0.125, 0.59375, 0.78125, 0.59375, -0.875, -0.3125, 0.125, 0.9375, 0.09375, 0.28125, 0.375, 0.15625, -0.65625, 0.8125, 0.53125, -0.53125, -0.375, -0.125, -0.625, -0.6875, 0.46875, 0.96875, -0.1875, 0.0, 0.5, -0.90625, -0.71875, 0.25, 0.84375, -0.8125, -0.65625, -0.78125, 0.65625, 0.28125, 0.34375, 0.46875, -0.6875, -0.375, -0.875, -0.59375, -0.5, 0.65625, 0.375, 0.65625, -0.375, 0.90625, -0.6875, -0.875, -0.46875, -0.53125, -0.40625, 0.375, -0.25, -0.71875, -0.90625, -0.375, 0.65625, -1.0, -0.21875, 0.46875, -0.5625, 0.0625, -0.5625, 0.15625, 0.21875, -0.90625, 0.59375, 0.8125, 0.9375, -0.875, 0.53125, -0.96875, -0.03125, -0.09375, 0.65625, 0.8125, 0.6875, -0.25, -0.0625, -0.53125, -0.15625, -0.375, -1.0, -0.28125, -0.875, 0.9375, 0.15625, -0.125, -0.125, -0.4375, 0.25, 0.09375, 0.75, 0.0625, -0.3125, -0.9375, 0.625, -0.09375, -0.46875, -0.65625, -0.28125, -0.1875, 0.1875, -0.4375, 0.3125, 0.84375, -1.0, 0.71875, -0.625, -0.21875, 0.53125, -0.34375, -0.75, 0.28125, -0.875, 0.28125, 0.625, -0.0625, -0.3125, -1.0, -0.0625, -0.53125, -0.6875, -0.3125, 0.375, 0.15625, -0.09375, -0.0625, 0.9375, -0.5, -0.28125, -0.625, -0.3125, 0.375, -0.3125, -0.96875, -0.15625, 0.3125, 0.25, 0.75, 0.5625, -0.59375, -0.96875, -0.28125, -0.40625, -0.84375, 0.3125, -0.90625, -0.625, -0.15625, -0.9375, -1.0, 0.90625, -0.78125, -0.25, -0.71875, 0.21875, 0.0625, 0.78125, -0.03125, -0.4375, -0.65625, -1.0, -0.96875, 0.1875, -0.40625, 0.03125, -0.59375, -0.0625, 0.4375, -0.125, 0.125, 0.6875, -0.90625, -0.28125, -0.90625, 0.34375, -0.75, 0.46875, -0.90625, 0.34375, -0.15625, 0.875, -0.15625, -0.6875, 0.625, 0.90625, 0.125, -0.71875, 0.65625, 0.71875, 0.59375, -0.53125, -0.25, -0.4375, 0.9375, 0.84375, 0.78125, 0.59375, 0.375, 0.0, 0.75, -0.5625, -0.9375, -0.125, 0.90625, -0.4375, 0.90625, 0.5, -0.09375, -0.96875, 0.28125, -0.0625, 0.0, 0.25, 0.59375, -0.875, -1.0, -0.78125, -0.59375, -0.40625, 0.84375, -0.8125, 0.84375, -0.875, 0.25, 0.34375, 0.46875, -0.1875, -0.6875, 0.40625, -0.53125, 0.6875, -0.59375, -0.96875, -0.125, -0.90625, 0.75, -0.09375, -0.53125, -0.8125, 0.1875, 0.53125, -0.875, -1.0, 0.90625, 0.9375, 0.90625, 0.0, 0.71875, 0.3125, 0.3125, 0.5, -0.34375, -0.15625, 0.6875, 0.90625, 0.1875, -0.375, 0.0625, -0.6875, 0.40625, -0.9375, -0.875, 0.09375, -0.40625, 0.75, 0.96875, 0.34375, -0.65625, 0.5, -0.375, -0.03125, -0.6875, -0.3125, 0.125, 0.8125, 0.6875, 0.21875, 0.25, 0.9375, 0.9375, 0.78125, -0.28125, -0.4375, 0.1875, 0.5625, -0.15625, -0.125, -0.5625, 0.34375, 0.21875, -0.09375, 0.375, 0.03125, -0.90625, -0.9375, 0.9375, -1.0, 0.53125, 0.375, 0.25, 0.40625, -0.625, -0.5, 0.53125, -0.21875, -0.25, -0.8125, -0.34375, -0.4375, -0.09375, -0.28125, 0.90625, -0.90625, -0.1875, -0.78125, -0.59375, -0.4375, 0.6875, 0.09375, -0.96875, -0.9375, 0.78125, 0.34375, 0.8125, -0.21875, 0.375, -0.3125, 0.65625, -0.0625, -0.65625, 0.03125, -0.53125, -0.34375, 0.96875, 0.4375, 0.65625, -0.1875, 0.96875, -0.3125, 0.53125, 0.03125, 0.4375, 0.0625, 0.21875, -0.5, 0.0, -0.59375, 0.90625, -0.1875, -0.125, -0.78125, 0.3125, -0.3125, 0.3125, -0.96875, 0.4375, 0.8125, 0.78125, -0.34375, -0.09375, 0.125, 0.15625, -0.21875, 0.96875, 0.0, 0.6875, -0.84375, 0.65625, -0.3125, -0.9375, -0.34375, 0.4375, 0.09375, -0.53125], [0.125, -0.59375, -0.875, -0.21875, -0.875, 0.03125, -0.78125, 0.5, -0.15625, 0.53125, -0.34375, -0.625, 0.3125, 0.71875, 0.65625, 0.4375, -0.5, 0.15625, 0.1875, 0.75, 0.03125, 0.8125, -0.5625, -0.5, 0.84375, -0.28125, -0.9375, -0.1875, -0.75, -0.90625, -0.78125, 0.34375, 0.9375, 0.625, -0.28125, 0.1875, 0.96875, -0.625, 0.21875, 0.46875, 0.0625, 0.4375, -0.75, 0.6875, 0.21875, -0.78125, -0.09375, 0.5, 0.15625, -0.90625, -0.28125, 0.4375, 0.96875, 0.5, 0.875, 0.09375, 0.46875, 0.46875, 0.875, -0.40625, -0.3125, 0.5625, 0.21875, 0.5, -0.09375, -0.5625, -0.78125, 0.15625, 0.28125, -0.96875, 0.21875, -0.8125, 0.96875, 0.65625, -0.6875, -0.28125, -0.8125, 0.84375, -0.875, 0.375, -0.96875, -0.21875, -0.1875, 0.46875, 0.71875, -0.8125, 0.84375, 0.3125, 0.875, -0.71875, -0.15625, 0.6875, -0.40625, -0.84375, 0.71875, 0.125, 0.125, -0.4375, 0.125, -0.3125, -0.59375, -0.75, 0.8125, -0.28125, 0.28125, 0.5, 0.03125, 0.25, 0.71875, 0.15625, 0.3125, -0.125, -0.65625, -0.75, 0.96875, -0.78125, 0.25, 0.25, 0.75, -0.15625, 0.46875, 0.78125, -0.21875, -0.0625, 0.375, 0.09375, 0.3125, 0.53125, -0.46875, 0.4375, -0.3125, -0.28125, 0.03125, -0.6875, 0.75, -0.96875, 0.125, 0.59375, 0.78125, 0.59375, -0.875, -0.3125, 0.125, 0.9375, 0.09375, 0.28125, 0.375, 0.15625, -0.65625, 0.8125, 0.53125, -0.53125, -0.375, -0.125, -0.625, -0.6875, 0.46875, 0.96875, -0.1875, 0.0, 0.5, -0.90625, -0.71875, 0.25, 0.84375, -0.8125, -0.65625, -0.78125, 0.65625, 0.28125, 0.34375, 0.46875, -0.6875, -0.375, -0.875, -0.59375, -0.5, 0.65625, 0.375, 0.65625, -0.375, 0.90625, -0.6875, -0.875, -0.46875, -0.53125, -0.40625, 0.375, -0.25, -0.71875, 0.4375, -0.375, 0.65625, -1.0, -0.21875, 0.46875, -0.5625, 0.0625, -0.5625, 0.15625, 0.21875, -0.90625, 0.59375, 0.8125, 0.9375, -0.875, 0.53125, -0.96875, -0.03125, -0.09375, 0.65625, 0.8125, 0.6875, -0.25, -0.0625, -0.53125, -0.15625, -0.375, -1.0, -0.28125, -0.875, 0.9375, 0.15625, -0.125, -0.125, -0.4375, 0.25, 0.09375, 0.75, 0.0625, -0.3125, -0.9375, 0.625, -0.09375, -0.46875, -0.65625, -0.28125, -0.1875, 0.1875, -0.4375, 0.3125, 0.84375, -1.0, 0.71875, -0.625, -0.21875, -0.875, -0.34375, -0.75, 0.28125, -0.875, 0.28125, 0.625, -0.0625, -0.3125, -1.0, -0.0625, -0.53125, -0.6875, -0.3125, 0.375, 0.15625, -0.09375, -0.0625, 0.9375, -0.5, -0.28125, -0.625, -0.3125, 0.375, -0.3125, -0.96875, -0.15625, 0.3125, 0.25, 0.75, 0.5625, -0.59375, -0.96875, -0.28125, -0.40625, -0.84375, 0.3125, -0.90625, -0.625, -0.15625, -0.9375, -1.0, 0.90625, -0.78125, -0.25, -0.71875, 0.21875, 0.0625, 0.78125, -0.03125, -0.4375, -0.0625, -1.0, -0.96875, 0.1875, -0.40625, 0.03125, -0.59375, -0.0625, 0.4375, -0.125, 0.125, 0.6875, -0.90625, -0.28125, -0.90625, 0.34375, -0.75, 0.46875, -0.90625, 0.34375, -0.15625, 0.875, -0.15625, -0.6875, 0.625, 0.90625, 0.125, -0.71875, 0.65625, 0.71875, 0.59375, -0.53125, -0.25, -0.4375, 0.9375, 0.84375, 0.78125, 0.59375, 0.375, 0.0, 0.75, -0.5625, -0.9375, -0.125, 0.90625, -0.4375, 0.90625, 0.5, -0.09375, -0.96875, 0.28125, -0.0625, 0.0, 0.25, 0.59375, -0.875, -1.0, -0.78125, -0.59375, -0.40625, 0.84375, -0.8125, 0.84375, -0.875, 0.25, 0.34375, 0.46875, -0.1875, -0.6875, 0.40625, -0.53125, 0.6875, -0.59375, -0.96875, -0.125, -0.90625, 0.75, -0.09375, -0.53125, -0.8125, 0.1875, 0.53125, -0.875, -1.0, 0.90625, 0.9375, 0.90625, 0.0, 0.71875, 0.3125, 0.3125, 0.5, -0.34375, -0.15625, 0.6875, 0.90625, 0.1875, -0.375, 0.0625, -0.6875, 0.40625, -0.9375, -0.875, 0.09375, -0.40625, 0.75, 0.96875, 0.34375, -0.65625, 0.5, -0.375, -0.03125, -0.6875, -0.3125, 0.125, 0.8125, 0.6875, 0.21875, 0.25, 0.9375, 0.9375, 0.78125, -0.28125, -0.4375, 0.1875, 0.5625, -0.15625, -0.125, -0.5625, 0.34375, 0.21875, -0.09375, 0.375, 0.03125, -0.90625, -0.9375, 0.9375, -1.0, 0.53125, 0.375, 0.25, 0.40625, -0.625, -0.5, 0.53125, -0.21875, -0.25, -0.8125, -0.34375, -0.4375, -0.09375, -0.28125, 0.90625, -0.90625, -0.1875, -0.78125, -0.59375, -0.4375, 0.6875, 0.09375, -0.96875, -0.9375, 0.78125, 0.34375, 0.8125, -0.21875, 0.375, -0.3125, 0.65625, -0.0625, -0.65625, 0.03125, -0.53125, -0.34375, 0.96875, 0.4375, 0.65625, -0.1875, 0.96875, -0.3125, 0.53125, 0.03125, 0.4375, 0.0625, 0.21875, -0.5, 0.0, -0.59375, 0.90625, -0.1875, -0.125, -0.78125, 0.3125, -0.3125, 0.3125, -0.96875, 0.4375, 0.8125, 0.78125, -0.34375, -0.09375, 0.125, 0.15625, -0.21875, 0.96875, 0.0, 0.6875, -0.84375, 0.65625, -0.3125, -0.9375, -0.34375, 0.4375, 0.09375, -0.53125], [0.125, -0.59375, -0.875, -0.21875, -0.875, 0.03125, -0.78125, 0.5, -0.15625, 0.53125, -0.34375, -0.625, 0.3125, 0.71875, 0.65625, 0.4375, -0.5, 0.15625, 0.1875, 0.75, 0.03125, 0.8125, -0.5625, -0.5, 0.84375, -0.28125, -0.9375, -0.1875, -0.75, -0.90625, -0.78125, 0.34375, 0.9375, 0.625, -0.28125, 0.1875, 0.96875, -0.625, 0.21875, 0.46875, 0.0625, 0.4375, -0.75, 0.6875, 0.21875, -0.78125, -0.09375, 0.5, 0.15625, -0.90625, -0.28125, 0.4375, 0.96875, 0.5, 0.875, 0.09375, 0.46875, 0.46875, 0.875, -0.40625, -0.3125, 0.5625, 0.21875, 0.5, -0.09375, -0.5625, -0.78125, 0.15625, 0.28125, -0.96875, 0.21875, -0.8125, 0.96875, 0.65625, -0.6875, -0.28125, -0.8125, 0.84375, -0.875, 0.375, -0.96875, -0.21875, -0.1875, 0.46875, 0.71875, -0.8125, 0.84375, 0.3125, 0.875, -0.71875, -0.15625, 0.6875, -0.40625, -0.84375, 0.71875, 0.125, 0.125, -0.4375, 0.125, -0.3125, -0.59375, -0.75, 0.8125, -0.28125, 0.28125, 0.5, 0.03125, 0.25, 0.71875, 0.15625, 0.3125, -0.125, -0.65625, -0.75, 0.96875, -0.78125, 0.25, 0.25, 0.75, -0.15625, 0.46875, 0.78125, -0.21875, -0.0625, 0.375, 0.09375, 0.3125, 0.53125, -0.46875, 0.4375, -0.3125, -0.28125, 0.03125, -0.6875, 0.75, -0.96875, 0.125, 0.59375, 0.78125, 0.59375, -0.875, -0.3125, 0.125, 0.9375, 0.09375, 0.28125, 0.375, 0.15625, -0.65625, 0.8125, 0.53125, -0.53125, -0.375, -0.125, -0.625, -0.6875, 0.46875, 0.96875, -0.1875, 0.0, 0.5, -0.90625, -0.71875, 0.25, 0.84375, -0.8125, -0.65625, -0.78125, 0.65625, 0.28125, 0.34375, 0.46875, -0.6875, -0.375, -0.875, -0.59375, -0.5, 0.65625, 0.375, 0.65625, -0.375, 0.90625, -0.6875, -0.875, -0.46875, -0.53125, -0.40625, 0.375, -0.25, -0.71875, 0.4375, -0.375, 0.65625, -1.0, -0.21875, 0.46875, -0.5625, 0.0625, -0.5625, 0.15625, 0.21875, -0.90625, 0.59375, 0.8125, 0.9375, -0.875, 0.53125, -0.96875, -0.03125, -0.09375, 0.65625, 0.8125, 0.6875, -0.25, -0.0625, -0.53125, -0.15625, -0.375, -1.0, -0.28125, -0.875, 0.9375, 0.15625, -0.125, -0.125, -0.4375, 0.25, 0.09375, 0.75, 0.0625, -0.3125, -0.9375, 0.625, -0.09375, -0.46875, -0.65625, -0.28125, -0.1875, 0.1875, -0.4375, 0.3125, 0.84375, -1.0, 0.71875, -0.625, -0.21875, -0.875, -0.34375, -0.75, 0.28125, -0.875, 0.28125, 0.625, -0.0625, -0.3125, -1.0, -0.0625, -0.53125, -0.6875, -0.3125, 0.375, 0.15625, -0.09375, -0.0625, 0.9375, -0.5, -0.28125, -0.625, -0.3125, 0.375, -0.3125, -0.96875, -0.15625, 0.3125, 0.25, 0.75, 0.5625, -0.59375, -0.96875, -0.28125, -0.40625, -0.84375, 0.3125, -0.90625, -0.625, -0.15625, -0.9375, -1.0, 0.90625, -0.78125, -0.25, -0.71875, 0.21875, -0.875, 0.78125, -0.03125, -0.4375, -0.0625, -1.0, -0.96875, 0.1875, -0.40625, 0.03125, -0.59375, -0.0625, 0.4375, -0.125, 0.125, 0.6875, -0.90625, -0.28125, -0.90625, 0.34375, -0.75, 0.46875, -0.90625, 0.34375, -0.15625, 0.875, -0.15625, -0.6875, 0.625, 0.90625, 0.125, -0.71875, 0.65625, 0.71875, 0.59375, -0.53125, -0.25, -0.4375, 0.9375, 0.84375, 0.78125, 0.59375, 0.375, 0.0, 0.75, -0.5625, -0.9375, -0.125, 0.90625, -0.4375, 0.90625, 0.5, -0.09375, -0.96875, 0.28125, -0.0625, 0.0, 0.25, 0.59375, -0.875, -1.0, -0.78125, -0.59375, -0.40625, 0.84375, -0.8125, 0.84375, -0.875, 0.25, 0.34375, 0.46875, -0.1875, -0.6875, 0.40625, -0.53125, 0.6875, -0.59375, -0.96875, -0.125, -0.90625, 0.75, -0.09375, -0.53125, -0.8125, 0.1875, 0.53125, -0.875, -1.0, 0.90625, 0.9375, 0.90625, 0.0, 0.71875, 0.3125, 0.3125, 0.5, -0.34375, -0.15625, 0.6875, 0.90625, 0.1875, -0.375, 0.0625, -0.6875, 0.40625, -0.9375, -0.875, 0.09375, -0.40625, 0.75, 0.96875, 0.34375, -0.65625, 0.5, -0.375, -0.03125, -0.6875, -0.3125, 0.125, 0.8125, 0.6875, 0.21875, 0.25, 0.9375, 0.9375, 0.78125, -0.28125, -0.4375, 0.1875, 0.5625, -0.15625, -0.125, -0.5625, 0.34375, 0.21875, -0.09375, 0.375, 0.03125, -0.90625, -0.9375, 0.9375, -1.0, 0.53125, 0.375, 0.25, 0.40625, -0.625, -0.5, 0.53125, -0.21875, -0.25, -0.8125, -0.34375, -0.4375, -0.09375, -0.28125, 0.90625, -0.90625, -0.1875, -0.78125, -0.59375, -0.4375, 0.6875, 0.09375, -0.96875, -0.9375, 0.78125, 0.34375, 0.8125, -0.21875, 0.375, -0.3125, 0.65625, -0.0625, -0.65625, 0.03125, -0.53125, -0.34375, 0.96875, 0.4375, 0.65625, -0.1875, 0.96875, -0.3125, 0.53125, 0.03125, 0.4375, 0.0625, 0.21875, -0.5, 0.0, -0.59375, 0.90625, -0.1875, -0.125, -0.78125, 0.3125, -0.3125, 0.3125, -0.96875, 0.4375, 0.8125, 0.78125, -0.34375, -0.09375, 0.125, 0.15625, -0.21875, 0.96875, 0.0, 0.6875, -0.84375, 0.65625, -0.3125, -0.9375, -0.34375, 0.4375, 0.09375, -0.53125], [0.125, -0.59375, -0.875, -0.21875, -0.875, 0.03125, -0.78125, 0.5, -0.15625, 0.53125, -0.34375, -0.625, 0.3125, 0.71875, 0.65625, 0.4375, -0.5, 0.15625, 0.1875, 0.75, 0.03125, 0.8125, -0.5625, -0.5, 0.84375, -0.28125, -0.9375, -0.1875, -0.75, -0.90625, -0.78125, 0.34375, 0.9375, 0.625, -0.28125, 0.1875, 0.96875, -0.625, 0.21875, 0.46875, 0.0625, 0.4375, -0.75, 0.6875, 0.21875, -0.78125, -0.09375, 0.5, 0.15625, -0.90625, -0.28125, 0.4375, 0.96875, 0.5, 0.875, 0.09375, 0.46875, 0.46875, 0.875, -0.40625, -0.3125, 0.5625, 0.21875, 0.5, -0.09375, -0.5625, -0.78125, 0.15625, 0.28125, -0.96875, 0.21875, -0.8125, 0.96875, 0.65625, -0.6875, -0.28125, -0.8125, 0.84375, -0.875, 0.375, -0.96875, -0.21875, -0.1875, 0.46875, 0.71875, -0.8125, 0.84375, 0.3125, 0.875, -0.71875, -0.15625, 0.6875, -0.40625, -0.84375, 0.71875, 0.125, 0.125, -0.4375, 0.125, -0.3125, -0.59375, -0.75, 0.8125, -0.28125, 0.28125, 0.5, 0.03125, 0.25, 0.71875, 0.15625, 0.3125, -0.125, -0.65625, -0.75, 0.96875, -0.78125, 0.25, 0.25, 0.75, -0.15625, 0.46875, 0.78125, -0.21875, -0.0625, 0.375, 0.09375, 0.3125, 0.53125, -0.46875, 0.4375, -0.3125, -0.28125, 0.03125, -0.6875, 0.75, -0.96875, 0.125, 0.59375, 0.78125, 0.59375, -0.875, -0.3125, 0.125, 0.9375, 0.09375, 0.28125, 0.375, 0.15625, -0.65625, 0.8125, 0.53125, -0.53125, -0.375, -0.125, -0.625, -0.6875, 0.46875, 0.96875, -0.1875, 0.0, 0.5, -0.90625, -0.71875, 0.25, 0.84375, -0.8125, -0.65625, -0.78125, 0.65625, 0.28125, 0.34375, 0.46875, -0.6875, -0.375, -0.875, -0.59375, -0.5, 0.65625, 0.375, 0.65625, -0.375, 0.90625, -0.6875, -0.875, -0.46875, -0.53125, -0.40625, 0.375, -0.25, -0.71875, 0.4375, -0.375, 0.65625, -1.0, -0.21875, 0.46875, -0.5625, 0.0625, -0.5625, 0.15625, 0.21875, -0.90625, 0.59375, 0.8125, 0.9375, -0.875, 0.53125, -0.96875, -0.03125, -0.09375, 0.65625, 0.8125, 0.6875, -0.25, -0.0625, -0.53125, -0.15625, -0.375, -1.0, -0.28125, -0.875, 0.9375, 0.15625, -0.125, -0.125, -0.4375, 0.25, 0.09375, 0.75, 0.0625, -0.3125, -0.9375, 0.625, -0.09375, -0.46875, -0.65625, -0.28125, -0.1875, 0.1875, -0.4375, 0.3125, 0.84375, -1.0, 0.71875, -0.625, -0.21875, -0.875, -0.34375, -0.75, 0.28125, -0.875, 0.28125, 0.625, -0.0625, -0.3125, -1.0, -0.0625, -0.53125, 0.34375, -0.3125, 0.375, 0.15625, -0.09375, -0.0625, 0.9375, -0.5, -0.28125, 0.21875, -0.3125, 0.375, -0.3125, -0.96875, -0.15625, 0.3125, 0.25, 0.75, 0.5625, -0.59375, -0.96875, -0.28125, -0.40625, -0.84375, 0.3125, -0.90625, -0.625, -0.15625, -0.9375, -1.0, 0.90625, -0.78125, -0.25, -0.71875, 0.21875, -0.875, 0.78125, -0.03125, -0.4375, -0.0625, -1.0, -0.96875, 0.1875, -0.40625, 0.03125, -0.59375, -0.0625, 0.4375, -0.125, 0.125, 0.6875, -0.90625, -0.28125, -0.90625, 0.34375, -0.75, 0.46875, -0.90625, 0.34375, -0.15625, 0.875, -0.15625, -0.6875, 0.625, 0.90625, 0.125, -0.71875, 0.65625, 0.71875, 0.59375, -0.53125, -0.25, -0.4375, 0.9375, 0.84375, 0.78125, 0.59375, 0.375, 0.0, 0.75, -0.5625, -0.9375, -0.125, 0.90625, -0.4375, 0.90625, 0.5, -0.09375, -0.96875, 0.28125, -0.0625, 0.0, 0.25, 0.59375, -0.875, -1.0, -0.78125, -0.59375, -0.40625, 0.84375, -0.8125, 0.84375, -0.875, 0.25, 0.34375, 0.46875, -0.1875, -0.6875, 0.40625, -0.53125, 0.6875, -0.59375, -0.96875, -0.125, -0.90625, 0.75, -0.09375, -0.53125, -0.8125, 0.1875, 0.53125, -0.875, -1.0, 0.90625, 0.9375, 0.90625, 0.0, 0.71875, 0.3125, 0.3125, 0.5, -0.34375, -0.15625, 0.6875, 0.90625, 0.1875, -0.375, 0.0625, -0.6875, 0.40625, -0.9375, -0.875, 0.09375, -0.40625, 0.75, 0.96875, 0.34375, -0.65625, 0.5, -0.375, -0.03125, -0.6875, -0.3125, 0.125, 0.8125, 0.6875, 0.21875, 0.25, 0.9375, 0.9375, 0.78125, -0.28125, -0.4375, 0.1875, 0.5625, -0.15625, -0.125, -0.5625, 0.34375, 0.21875, -0.09375, 0.375, 0.03125, -0.90625, -0.9375, 0.9375, -1.0, 0.53125, 0.375, 0.25, 0.40625, -0.625, -0.5, 0.53125, -0.21875, -0.25, -0.8125, -0.34375, -0.4375, -0.09375, -0.28125, 0.90625, -0.90625, -0.1875, -0.78125, -0.59375, -0.4375, 0.6875, 0.09375, -0.96875, -0.9375, 0.78125, 0.34375, 0.8125, -0.21875, 0.375, -0.3125, 0.65625, -0.0625, -0.65625, 0.03125, -0.53125, -0.34375, 0.96875, 0.4375, 0.65625, -0.1875, 0.96875, -0.3125, 0.53125, 0.03125, 0.4375, 0.0625, 0.21875, -0.5, 0.0, -0.59375, 0.90625, -0.1875, -0.125, -0.78125, 0.3125, -0.3125, 0.3125, -0.96875, 0.4375, 0.8125, 0.78125, -0.34375, -0.09375, 0.125, 0.15625, -0.21875, 0.96875, 0.0, 0.6875, -0.84375, 0.65625, -0.3125, -0.9375, -0.34375, 0.4375, 0.09375, -0.53125], [0.125, -0.59375, -0.875, -0.21875, -0.875, 0.03125, -0.78125, 0.5, -0.15625, 0.53125, -0.34375, -0.625, 0.3125, 0.71875, 0.65625, 0.4375, -0.5, 0.15625, 0.1875, 0.75, 0.03125, 0.8125, -0.5625, -0.5, 0.84375, -0.28125, -0.9375, -0.1875, -0.75, -0.90625, -0.78125, 0.34375, 0.9375, 0.625, -0.28125, 0.1875, 0.96875, -0.625, 0.21875, 0.46875, 0.0625, 0.4375, -0.75, 0.6875, 0.21875, -0.78125, -0.09375, 0.5, 0.15625, -0.90625, -0.28125, 0.4375, 0.96875, 0.5, 0.875, 0.09375, 0.46875, 0.46875, 0.875, -0.40625, -0.3125, 0.5625, 0.21875, 0.5, -0.09375, -0.5625, -0.78125, 0.15625, 0.28125, -0.96875, 0.21875, -0.8125, 0.96875, 0.65625, -0.6875, -0.28125, -0.8125, 0.84375, -0.875, 0.375, -0.96875, -0.21875, -0.1875, 0.46875, 0.71875, -0.8125, 0.84375, 0.3125, 0.875, -0.71875, -0.15625, 0.6875, -0.40625, -0.84375, 0.71875, 0.125, 0.8125, -0.4375, 0.125, -0.3125, -0.59375, -0.75, 0.8125, -0.28125, 0.28125, 0.5, 0.03125, 0.25, 0.71875, 0.15625, 0.3125, -0.125, -0.65625, -0.75, 0.96875, -0.78125, 0.25, 0.25, 0.75, -0.15625, 0.46875, 0.78125, -0.21875, -0.0625, 0.375, 0.09375, 0.3125, 0.53125, -0.46875, 0.4375, -0.3125, -0.28125, 0.03125, -0.6875, 0.75, -0.96875, 0.125, 0.59375, 0.78125, 0.59375, -0.875, -0.3125, 0.125, 0.9375, 0.09375, 0.28125, 0.375, 0.15625, -0.65625, 0.8125, 0.53125, -0.53125, -0.375, -0.125, -0.625, -0.6875, 0.46875, 0.96875, -0.1875, 0.0, 0.5, -0.90625, -0.71875, 0.25, 0.84375, -0.8125, -0.65625, -0.78125, 0.65625, 0.28125, 0.34375, 0.46875, -0.6875, -0.375, -0.875, -0.59375, -0.5, 0.65625, 0.375, 0.65625, -0.375, 0.90625, -0.6875, -0.875, -0.46875, -0.53125, -0.40625, 0.375, -0.25, -0.71875, 0.4375, -0.375, 0.65625, -1.0, -0.21875, 0.46875, -0.5625, 0.0625, -0.5625, 0.15625, 0.21875, -0.90625, 0.59375, 0.8125, 0.9375, -0.875, 0.53125, -0.96875, -0.03125, -0.09375, 0.65625, 0.8125, 0.6875, -0.25, -0.0625, -0.53125, -0.15625, -0.375, -1.0, -0.28125, -0.875, 0.9375, 0.15625, -0.125, -0.125, -0.4375, 0.25, 0.09375, 0.75, 0.0625, -0.3125, -0.9375, 0.625, -0.09375, -0.46875, -0.65625, -0.28125, -0.1875, 0.1875, -0.4375, 0.3125, 0.84375, -1.0, 0.71875, -0.625, -0.21875, -0.875, -0.34375, -0.75, 0.28125, -0.875, 0.28125, 0.625, -0.0625, -0.3125, -1.0, -0.0625, -0.53125, 0.34375, -0.3125, 0.375, 0.15625, -0.09375, -0.0625, 0.9375, -0.5, -0.28125, 0.21875, -0.3125, 0.375, -0.3125, -0.96875, -0.15625, 0.3125, 0.25, 0.75, 0.5625, -0.59375, -0.96875, -0.28125, -0.40625, -0.84375, 0.3125, -0.90625, -0.625, -0.15625, -0.9375, -1.0, 0.90625, -0.78125, -0.25, -0.71875, 0.21875, -0.875, 0.78125, -0.03125, -0.4375, -0.0625, -1.0, -0.96875, 0.1875, -0.40625, 0.03125, -0.59375, -0.0625, 0.4375, -0.125, 0.125, 0.6875, -0.90625, -0.28125, -0.90625, 0.34375, 0.90625, 0.46875, -0.90625, 0.34375, -0.15625, 0.875, -0.15625, -0.6875, 0.625, 0.90625, 0.125, -0.71875, 0.65625, 0.71875, 0.59375, -0.53125, -0.25, -0.4375, 0.9375, 0.84375, 0.78125, 0.59375, 0.375, 0.0, 0.75, -0.5625, -0.9375, -0.125, 0.90625, -0.4375, 0.90625, 0.5, -0.09375, -0.96875, 0.28125, -0.0625, 0.0, 0.25, 0.59375, -0.875, -1.0, -0.78125, -0.59375, -0.40625, 0.84375, -0.8125, 0.84375, -0.875, 0.25, 0.34375, 0.46875, -0.1875, -0.6875, 0.40625, -0.53125, 0.6875, -0.59375, -0.96875, -0.125, -0.90625, 0.75, -0.09375, -0.53125, -0.8125, 0.1875, 0.53125, -0.875, -1.0, 0.90625, 0.9375, 0.90625, 0.0, 0.71875, 0.3125, 0.3125, 0.5, -0.34375, -0.15625, 0.6875, 0.90625, 0.1875, -0.375, 0.0625, -0.6875, 0.40625, -0.9375, -0.875, 0.09375, -0.40625, 0.75, 0.96875, 0.34375, -0.65625, 0.5, -0.375, -0.03125, -0.6875, -0.3125, 0.125, 0.8125, 0.6875, 0.21875, 0.25, 0.9375, 0.9375, 0.78125, -0.28125, -0.4375, 0.1875, 0.5625, -0.15625, -0.125, -0.5625, 0.34375, 0.21875, -0.09375, 0.375, 0.03125, -0.90625, -0.9375, 0.9375, -1.0, 0.53125, 0.375, 0.25, 0.40625, -0.625, -0.5, 0.53125, -0.21875, -0.25, -0.8125, -0.34375, -0.4375, -0.09375, -0.28125, 0.90625, -0.90625, -0.1875, -0.78125, -0.59375, -0.4375, 0.6875, 0.09375, -0.96875, -0.9375, 0.78125, 0.34375, 0.8125, -0.21875, 0.375, -0.3125, 0.65625, -0.0625, -0.65625, 0.03125, -0.53125, -0.34375, 0.96875, 0.4375, 0.65625, -0.1875, 0.96875, -0.3125, 0.53125, 0.03125, 0.4375, 0.0625, 0.21875, -0.5, 0.0, -0.59375, 0.90625, -0.1875, -0.125, -0.78125, 0.3125, -0.3125, 0.3125, -0.96875, 0.4375, 0.8125, 0.78125, -0.34375, -0.09375, 0.125, 0.15625, -0.21875, 0.96875, 0.0, 0.6875, -0.84375, 0.65625, -0.3125, -0.9375, -0.34375, 0.4375, 0.09375, -0.53125], [0.125, -0.59375, -0.875, -0.21875, -0.875, 0.03125, -0.78125, 0.5, -0.15625, 0.53125, -0.34375, -0.625, 0.3125, 0.71875, 0.65625, 0.4375, -0.5, 0.15625, 0.1875, 0.75, 0.03125, 0.8125, -0.5625, -0.5, 0.84375, -0.28125, -0.9375, -0.1875, -0.75, -0.90625, -0.78125, 0.34375, 0.9375, 0.625, -0.28125, 0.1875, 0.96875, -0.625, 0.21875, 0.46875, 0.0625, 0.4375, -0.75, 0.6875, 0.21875, -0.78125, -0.09375, 0.5, 0.15625, -0.90625, -0.28125, 0.4375, 0.96875, 0.5, 0.875, 0.09375, 0.46875, 0.46875, 0.875, -0.40625, -0.3125, 0.5625, 0.21875, 0.5, -0.09375, -0.5625, -0.78125, 0.15625, 0.28125, -0.96875, 0.21875, -0.8125, 0.96875, 0.65625, -0.6875, -0.28125, -0.8125, 0.84375, -0.875, 0.375, -0.96875, -0.21875, -0.1875, 0.46875, 0.71875, -0.8125, 0.84375, 0.3125, 0.875, -0.71875, -0.15625, 0.6875, -0.40625, -0.84375, 0.71875, 0.125, 0.125, -0.4375, 0.125, -0.3125, -0.59375, -0.75, 0.8125, -0.28125, 0.28125, 0.5, 0.03125, 0.25, 0.71875, 0.15625, 0.3125, -0.125, -0.65625, -0.75, 0.96875, -0.78125, 0.25, 0.25, 0.75, -0.15625, 0.46875, -0.59375, -0.21875, -0.0625, 0.375, 0.09375, 0.3125, 0.53125, -0.46875, 0.4375, -0.3125, -0.28125, 0.03125, -0.6875, 0.75, -0.96875, 0.125, 0.59375, 0.78125, 0.59375, -0.875, -0.3125, 0.125, 0.9375, 0.09375, 0.28125, 0.375, 0.15625, -0.65625, 0.8125, 0.53125, -0.53125, -0.375, -0.125, -0.625, -0.6875, 0.46875, 0.96875, -0.1875, 0.0, 0.5, -0.90625, -0.71875, 0.25, 0.84375, -0.8125, -0.65625, -0.78125, 0.65625, 0.28125, 0.34375, 0.46875, -0.6875, -0.375, -0.875, -0.59375, -0.5, 0.65625, 0.375, 0.65625, -0.375, 0.90625, -0.6875, -0.875, -0.46875, -0.53125, -0.40625, 0.375, -0.25, -0.71875, 0.4375, -0.375, 0.65625, -1.0, -0.21875, 0.46875, -0.5625, 0.0625, -0.5625, 0.15625, 0.21875, -0.90625, 0.59375, 0.8125, 0.9375, -0.875, 0.53125, -0.96875, -0.03125, -0.09375, 0.65625, 0.8125, 0.6875, -0.25, -0.0625, -0.53125, -0.15625, -0.375, -1.0, -0.28125, -0.875, 0.9375, 0.15625, -0.125, -0.125, -0.4375, 0.25, 0.09375, 0.75, 0.0625, -0.3125, -0.9375, 0.625, -0.09375, -0.46875, -0.65625, -0.28125, -0.1875, 0.1875, -0.4375, 0.3125, 0.84375, -1.0, 0.71875, -0.625, -0.21875, -0.875, -0.34375, -0.75, 0.28125, -0.875, 0.28125, 0.625, -0.0625, -0.3125, -1.0, -0.0625, -0.53125, 0.34375, -0.3125, 0.375, 0.15625, -0.09375, -0.0625, 0.9375, -0.5, -0.28125, 0.21875, -0.3125, 0.375, -0.3125, -0.96875, -0.15625, 0.3125, 0.25, 0.75, 0.5625, -0.59375, -0.96875, -0.28125, -0.40625, -0.84375, 0.3125, -0.90625, -0.625, -0.15625, -0.9375, -1.0, 0.90625, -0.78125, -0.25, -0.71875, 0.21875, -0.875, 0.78125, -0.03125, -0.4375, -0.0625, -1.0, -0.96875, 0.1875, -0.40625, 0.03125, -0.59375, -0.0625, 0.4375, -0.125, 0.125, 0.6875, -0.90625, -0.28125, -0.90625, 0.34375, 0.90625, 0.46875, -0.90625, 0.34375, -0.15625, 0.875, -0.15625, -0.6875, 0.625, 0.90625, 0.125, -0.71875, 0.65625, 0.71875, 0.59375, -0.53125, -0.25, -0.4375, 0.9375, 0.84375, 0.78125, 0.59375, 0.375, 0.0, 0.75, -0.5625, -0.9375, -0.125, 0.90625, -0.4375, 0.90625, 0.5, -0.09375, -0.96875, 0.28125, -0.0625, 0.0, 0.25, 0.59375, -0.875, -1.0, -0.78125, -0.59375, -0.40625, 0.84375, -0.8125, 0.84375, -0.875, 0.25, 0.34375, 0.46875, -0.1875, -0.6875, 0.40625, -0.53125, 0.6875, -0.59375, -0.96875, -0.125, -0.90625, 0.75, -0.09375, -0.53125, -0.8125, 0.1875, 0.53125, -0.875, -1.0, 0.90625, 0.9375, 0.90625, 0.0, 0.71875, 0.3125, 0.3125, 0.5, -0.34375, -0.15625, 0.6875, 0.90625, 0.1875, -0.375, 0.0625, -0.6875, 0.40625, -0.9375, -0.875, 0.09375, -0.40625, 0.75, 0.96875, 0.34375, -0.65625, 0.5, -0.375, -0.03125, -0.6875, -0.3125, 0.125, 0.8125, 0.6875, 0.21875, 0.25, 0.9375, 0.9375, 0.78125, -0.28125, -0.4375, 0.1875, 0.5625, -0.15625, -0.125, -0.5625, 0.34375, 0.21875, -0.09375, 0.375, 0.03125, -0.90625, -0.9375, 0.9375, -1.0, 0.53125, 0.375, 0.25, 0.40625, -0.625, -0.5, 0.53125, -0.21875, -0.25, -0.8125, -0.34375, -0.4375, -0.09375, -0.28125, 0.90625, -0.90625, -0.1875, -0.78125, -0.59375, -0.4375, 0.6875, 0.09375, -0.96875, -0.9375, 0.78125, 0.34375, 0.8125, -0.21875, 0.375, -0.3125, 0.65625, -0.0625, -0.65625, 0.03125, -0.53125, -0.34375, 0.96875, 0.4375, 0.65625, -0.1875, 0.96875, -0.3125, 0.0625, 0.03125, 0.4375, 0.0625, 0.21875, -0.5, 0.0, -0.59375, 0.90625, -0.1875, -0.125, -0.78125, 0.3125, -0.3125, 0.3125, -0.96875, 0.4375, 0.8125, 0.78125, -0.34375, -0.09375, 0.125, 0.15625, -0.21875, 0.96875, 0.0, 0.6875, -0.84375, 0.65625, -0.3125, -0.9375, -0.34375, 0.4375, 0.09375, -0.53125], [0.125, -0.59375, -0.875, -0.21875, -0.875, 0.03125, -0.78125, 0.5, -0.15625, 0.53125, -0.34375, -0.625, 0.3125, 0.71875, 0.65625, 0.4375, -0.5, 0.15625, 0.1875, 0.75, 0.03125, 0.8125, -0.5625, -0.5, 0.84375, -0.28125, -0.9375, -0.1875, -0.75, -0.90625, -0.78125, 0.34375, 0.9375, 0.625, -0.28125, 0.1875, 0.96875, -0.625, 0.21875, 0.46875, 0.0625, 0.4375, -0.75, 0.6875, 0.21875, -0.78125, -0.09375, 0.5, 0.15625, -0.90625, -0.28125, 0.4375, 0.96875, 0.5, 0.875, 0.09375, 0.46875, 0.46875, 0.875, -0.40625, -0.3125, 0.5625, 0.21875, 0.5, -0.09375, -0.5625, -0.78125, 0.15625, 0.28125, -0.96875, 0.21875, -0.8125, 0.96875, 0.65625, -0.6875, -0.28125, -0.8125, 0.84375, -0.875, 0.375, -0.96875, -0.21875, -0.1875, 0.46875, 0.71875, -0.8125, 0.84375, 0.3125, 0.875, -0.71875, -0.15625, 0.6875, -0.40625, -0.84375, 0.71875, 0.125, 0.125, -0.4375, 0.125, -0.3125, -0.59375, -0.75, 0.8125, -0.28125, 0.28125, 0.5, 0.03125, 0.25, 0.71875, 0.15625, 0.3125, -0.125, -0.65625, -0.75, 0.96875, -0.78125, 0.25, 0.25, 0.75, -0.15625, 0.46875, 0.78125, -0.21875, -0.0625, 0.375, 0.09375, 0.3125, 0.53125, -0.46875, 0.4375, -0.3125, -0.28125, 0.03125, -0.6875, 0.75, -0.96875, 0.125, 0.59375, 0.78125, 0.59375, -0.875, -0.3125, 0.125, 0.9375, 0.09375, 0.28125, 0.375, 0.15625, -0.65625, 0.8125, 0.53125, -0.53125, -0.375, -0.125, -0.625, -0.6875, 0.46875, 0.96875, -0.1875, 0.0, 0.5, -0.90625, -0.71875, 0.25, 0.84375, -0.8125, -0.65625, -0.78125, 0.65625, 0.28125, 0.34375, 0.46875, -0.6875, -0.375, -0.875, -0.59375, -0.5, 0.65625, 0.375, 0.65625, -0.375, 0.90625, -0.6875, -0.875, -0.46875, -0.53125, -0.40625, 0.375, -0.25, -0.71875, 0.4375, -0.375, 0.65625, -1.0, -0.21875, 0.46875, -0.5625, 0.0625, -0.5625, 0.15625, 0.21875, -0.90625, 0.59375, 0.8125, 0.9375, -0.875, 0.53125, -0.96875, -0.03125, -0.09375, 0.65625, 0.8125, 0.6875, -0.25, -0.0625, -0.53125, -0.15625, -0.375, -1.0, -0.28125, -0.875, 0.9375, 0.15625, -0.125, -0.125, -0.4375, 0.25, 0.09375, 0.75, 0.0625, -0.3125, -0.9375, -0.5625, -0.09375, -0.46875, -0.65625, -0.28125, -0.1875, 0.1875, -0.4375, 0.3125, 0.84375, -1.0, 0.71875, -0.625, -0.21875, -0.875, -0.34375, -0.75, 0.28125, -0.875, 0.28125, 0.625, -0.0625, -0.3125, -1.0, -0.0625, -0.53125, 0.34375, -0.3125, 0.375, 0.15625, -0.09375, -0.0625, 0.9375, -0.5, -0.28125, 0.21875, -0.3125, 0.375, -0.3125, -0.96875, -0.15625, 0.3125, 0.25, 0.75, 0.5625, -0.59375, -0.96875, -0.28125, -0.40625, -0.84375, 0.3125, -0.90625, -0.625, -0.15625, -0.9375, -1.0, 0.90625, -0.78125, -0.25, -0.71875, 0.21875, -0.875, 0.78125, -0.03125, -0.4375, -0.0625, -1.0, -0.96875, 0.1875, -0.40625, 0.03125, -0.59375, -0.0625, 0.4375, -0.125, 0.125, 0.6875, -0.90625, -0.28125, -0.90625, 0.34375, 0.90625, 0.46875, -0.90625, 0.34375, -0.15625, 0.875, -0.15625, -0.6875, 0.625, 0.90625, 0.125, -0.71875, 0.65625, 0.71875, 0.59375, -0.53125, -0.25, -0.4375, 0.9375, 0.84375, 0.78125, 0.59375, 0.375, 0.0, 0.75, -0.5625, -0.9375, -0.125, 0.90625, -0.4375, 0.90625, 0.5, -0.09375, -0.96875, 0.28125, -0.0625, 0.0, 0.25, 0.59375, -0.875, -1.0, -0.78125, -0.59375, -0.40625, 0.84375, -0.8125, 0.84375, -0.875, 0.25, 0.34375, 0.46875, -0.1875, -0.6875, 0.40625, -0.53125, 0.6875, -0.59375, -0.96875, -0.125, -0.90625, 0.75, -0.09375, -0.53125, -0.8125, 0.1875, 0.53125, -0.875, -1.0, 0.90625, 0.9375, 0.90625, 0.0, 0.71875, 0.3125, 0.3125, 0.5, -0.34375, -0.15625, 0.6875, 0.90625, 0.1875, -0.375, 0.0625, -0.6875, 0.40625, -0.9375, -0.875, 0.09375, -0.40625, 0.75, 0.96875, 0.34375, -0.65625, 0.5, -0.375, -0.03125, -0.6875, -0.3125, 0.125, 0.8125, 0.6875, 0.21875, 0.25, 0.9375, 0.9375, 0.78125, -0.28125, -0.4375, 0.1875, 0.5625, -0.15625, -0.125, -0.5625, 0.34375, 0.21875, -0.09375, 0.375, 0.03125, -0.90625, -0.9375, 0.9375, -1.0, 0.53125, 0.375, 0.25, 0.40625, -0.625, -0.5, 0.53125, -0.21875, -0.25, -0.8125, -0.34375, -0.4375, -0.09375, -0.28125, 0.90625, -0.90625, -0.1875, -0.78125, -0.59375, -0.96875, 0.6875, 0.09375, -0.96875, -0.9375, 0.78125, 0.34375, 0.8125, -0.21875, 0.375, -0.3125, 0.65625, -0.0625, -0.65625, 0.03125, -0.53125, -0.34375, 0.96875, 0.4375, 0.65625, -0.1875, 0.96875, -0.3125, 0.0625, 0.03125, 0.4375, 0.0625, 0.21875, -0.5, 0.0, -0.59375, 0.90625, -0.1875, -0.125, -0.78125, 0.1875, -0.3125, 0.3125, -0.96875, 0.4375, 0.8125, 0.78125, -0.34375, -0.09375, 0.125, 0.15625, -0.21875, 0.96875, 0.0, 0.6875, -0.84375, 0.65625, -0.3125, -0.9375, -0.34375, 0.4375, 0.09375, -0.53125]]

In [0]:
#@title Generate samples from the prior.

seq = mel_2bar.decode(genValues, 32)
for ns in seq:
  play(ns)

In [0]:
for i, ns in enumerate(seq):
  download(ns, 'mel_2bar_GA_%d.mid' % i)